[矢吹太朗『Webのしくみ』（サイエンス社, 2020）](https://github.com/taroyabuki/webbook)

下のアイコンをクリックすれば，この文書に掲載されているコードを，Google Colab上で実行できます．（Googleのアカウントが必要です．）

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/taroyabuki/webbook/blob/master/chapters/04_thompson.ipynb)

# C言語のプログラムの実行方法・トンプソンハック

## hello, world

「hello, world」と表示するプログラムのC言語のコードを書く。

In [ ]:
%%writefile hello.c
#include <stdio.h>
int main() {
  printf("hello, world\n");
}

コンパイルする。

In [ ]:
!gcc hello.c

実行する。

In [ ]:
!./a.out

教科書の図4.3の再現

In [ ]:
!gcc hello.c
!od -An -tx a.out

## Quine（トンプソンハックのための準備）

> どんなコンピュータにも絶対の信頼を置くべきではないというのが本当で，重要なことをオンライン投票で決めるのは，途方もなく無謀で危険なことなのである。これは重大な主張であり，即刻懐疑的な反応が出てきそうだ。&mdash;しかし，それは私たちの目的には好都合だ。オンライン投票またはコンピュータによる投票は安全で，簡単で，便利だと思う人が多くなるほど，あなたは彼らの国家を盗みやすくなるのだから。
>
> じゃあ，がんばってやってみよう！

- <img src="https://ndlsearch.ndl.go.jp/thumbnail/9784152102553.jpg" alt="科学でかなえる世界征服"> ライアン・ノース『科学でかなえる世界征服』（早川書房, 2023）
  1. 「ログイン」のコード→（悪のコンパイラのコード→の悪のコンパイラの実行形式）→「バックドアを仕込まれたログイン」　**「ログイン」のコードからは悪事を検出できない。**
  1. 「善いコンパイラ」のコード→（「悪のコンパイラ」の実行形式）→「悪のコンパイラ」　**「善いコンパイラ」のソースコードからは悪事を検出できない。**
  1. 電子投票システムを「悪のコンパイラ」のターゲットにする。
  1. “選挙の必須条件：開かれており，厳重に警備されており，匿名性が保て，透明で，正確”
  1. “私たちが実生活で出くわすたいていのこと&mdash;は，この必要条件のすべてを満たしているわけではないか，あるいは，満たしていたとすればそれは，大して重要ではないかのいずれかである。”
    - クレジットカード？　詐欺に対して補償してもそれで信頼してもらえるならよい。
    - オンラインバンキングも同じ。
  1. “選挙が全体として攻撃された場合は，そういうよくある低リスク事例の1つではない。選挙は，ハッキングが，修復不可能な大きな影響を及ぼし得るケースだ。なぜなら，終わってしまったあとで，ある選挙全体の最終結果を修正するのは容易ではないからだ。”
  1. 似た攻撃：
    - イランのウラン濃縮用遠心分離機に対する攻撃（2010）
    - OpenSSLの「ハートブリード」バグ（2011）
    - フォルクスワーゲンの車両に搭載されているコンピュータの不正（2015）
  1. ブロックチェーン？　匿名性が破られやすい。
  1. 紙の投票用紙？　“1人の悪人の行為で与えられる損傷の大きさが制限されている。
  1. オンライン投票の導入例：
    - スイス：オンライン投票ができるのは有権者の10%
    - フランス：2003年に開始，2017年にセキュリティ上の懸念で中断
    - ドイツ：2000年に実証実験開始，2009年に停止（一般市民がコードを理解できないため）

> 教訓は明白だ。自分で完全に作成していないコードは信用してはいけない。（特に私のような人間を雇っている会社のコードは。）いくらソース・レベルでの検証や精査を行っても、信頼できないコードからあなたを守ることはできない。（ケン・トンプソン）

- トンプソンハック
  - [Ken Thompson. Reflections on trusting trust. <em>Communications of the ACM</em>, Vol. 27, No. 8, pp. 761&ndash;763, August 1984.](https://dl.acm.org/doi/10.1145/358198.358210)
  - [Yasunori Kuji訳「信頼を信頼することについての考察」](https://qiita.com/uturned0/items/95dad5cd688c3c9f5df9)
  - [https://www.google.com/search?q=トンプソン・ハック](https://www.google.com/search?q=%E3%83%88%E3%83%B3%E3%83%97%E3%82%BD%E3%83%B3%E3%83%BB%E3%83%8F%E3%83%83%E3%82%AF)
  - 対策の例：[oriansj/stage0](https://github.com/oriansj/stage0)
    - https://x.com/EzoeRyou/status/1429720956847595520
- ハードウェアも信用できない。例：レバノンのポケベル爆発（2024）



説明のためのQuineを作る。（トンプソンのQuineは後で示す。）

> 知られている限り最も単純なANSI CのQuine。コンパイラの警告が出ないもの

Geminiの結果を示す。

In [ ]:
%%writefile quine.c
#include <stdio.h>
const char *s = "#include <stdio.h>%cconst char *s = %c%s%c;%cint main() { printf(s, 10, 34, s, 34, 10, 10); return 0; }%c";
int main() { printf(s, 10, 34, s, 34, 10, 10); return 0; }

In [ ]:
!gcc quine.c
!./a.out

## ♠トンプソンのQuine

文献のコードがQuineになるように，AIを使って修正する。

```
★の部分を補ってQuineになるようにして。★のところ以外は変更しないで。出力をone.cとして，one.cがQuineになっていればよい。
char  s[] = {
	'\t',
	'0',
	'\n',
	'}',
	';',
	'\n',
	'\n',
	'/',
	'*',
	'\n',
  ★
	0
};

/*
 * The string s is a
 * representation of the body
 * of this program from '0'
 * to the end.
 */

main()
{
	int i;

	printf("char\ts[] = {\n");
	for(i=0; s[i]; i++)
		printf("\t%d,\n", s[i]);
	printf("%s", s);
}
```

Claudeの結果を示す。

In [ ]:
%%writefile quine.c
char  s[] = {
	'\t',
	'0',
	'\n',
	'}',
	';',
	'\n',
	'\n',
	'/',
	'*',
	'\n',
	' ',
	'*',
	' ',
	'T',
	'h',
	'e',
	' ',
	's',
	't',
	'r',
	'i',
	'n',
	'g',
	' ',
	's',
	' ',
	'i',
	's',
	' ',
	'a',
	'\n',
	' ',
	'*',
	' ',
	'r',
	'e',
	'p',
	'r',
	'e',
	's',
	'e',
	'n',
	't',
	'a',
	't',
	'i',
	'o',
	'n',
	' ',
	'o',
	'f',
	' ',
	't',
	'h',
	'e',
	' ',
	'b',
	'o',
	'd',
	'y',
	'\n',
	' ',
	'*',
	' ',
	'o',
	'f',
	' ',
	't',
	'h',
	'i',
	's',
	' ',
	'p',
	'r',
	'o',
	'g',
	'r',
	'a',
	'm',
	' ',
	'f',
	'r',
	'o',
	'm',
	' ',
	'\'',
	'0',
	'\'',
	'\n',
	' ',
	'*',
	' ',
	't',
	'o',
	' ',
	't',
	'h',
	'e',
	' ',
	'e',
	'n',
	'd',
	'.',
	'\n',
	' ',
	'*',
	'/',
	'\n',
	'\n',
	'm',
	'a',
	'i',
	'n',
	'(',
	')',
	'\n',
	'{',
	'\n',
	'\t',
	'i',
	'n',
	't',
	' ',
	'i',
	';',
	'\n',
	'\t',
	'\n',
	'\t',
	'p',
	'r',
	'i',
	'n',
	't',
	'f',
	'(',
	'"',
	'c',
	'h',
	'a',
	'r',
	'\\',
	't',
	's',
	'[',
	']',
	' ',
	'=',
	' ',
	'{',
	'\\',
	'n',
	'"',
	')',
	';',
	'\n',
	'\t',
	'f',
	'o',
	'r',
	'(',
	'i',
	'=',
	'0',
	';',
	' ',
	's',
	'[',
	'i',
	']',
	';',
	' ',
	'i',
	'+',
	'+',
	')',
	'\n',
	'\t',
	'\t',
	'p',
	'r',
	'i',
	'n',
	't',
	'f',
	'(',
	'"',
	'\\',
	't',
	'%',
	'd',
	',',
	'\\',
	'n',
	'"',
	',',
	' ',
	's',
	'[',
	'i',
	']',
	')',
	';',
	'\n',
	'\t',
	'p',
	'r',
	'i',
	'n',
	't',
	'f',
	'(',
	'"',
	'%',
	's',
	'"',
	',',
	' ',
	's',
	')',
	';',
	'\n',
	'}',
	0
};

/*
 * The string s is a
 * representation of the body
 * of this program from '0'
 * to the end.
 */

main()
{
	int i;

	printf("char\ts[] = {\n");
	for(i=0; s[i]; i++)
		printf("\t%d,\n", s[i]);
	printf("%s", s);
}

C言語のコードの形式が古いから，それを許容するためのオプションを設定してコンパイルする。実行結果をone.cとすると，one.cがQuineになっている。その実行結果をtwo.cとすると，one.cとtwo.cが同じになる。

In [ ]:
!gcc -Wno-implicit-function-declaration -Wno-builtin-declaration-mismatch -Wno-implicit-int quine.c
!./a.out > one.c
!gcc -Wno-implicit-function-declaration -Wno-builtin-declaration-mismatch -Wno-implicit-int one.c
!./a.out > two.c
!diff one.c two.c